In [2]:
pip install python-docx


     |████████████████████████████████| 5.5MB 5.0MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=5c0002462e98c42d46a1fcdf6ed57fe3c0af0f589c67e978cedc63d128343bc2
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx


In [3]:
pip install scipy


In [4]:
pip install mypy


     |████████████████████████████████| 21.6MB 6.1MB/s 
     |████████████████████████████████| 747kB 35.1MB/s 


In [5]:
pip install python-docx


In [6]:
pip install nltk


In [7]:
pip install tika


  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=cd6847c3920e505c7915dd5dbd2c2ec7e6a54340ea041c90e1775565d301ae3f
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [ ]:
pip install spacy

In [ ]:
pip install numpy

In [ ]:
pip install pandas 

In [ ]:
pip install gensim 

In [ ]:
pip install sklearn

In [ ]:
def customFilter(token):
    customized_stop_words = [
        "work",
        "experience",
        "education",
        "mobile",
        "email",
        "number",
        "num",
        "professional",
        "career",
        "history",
        "histories",
        "skill",
        "skills",
        "activity",
        "activities",
        "curriculum",
        "tool",
        "tools",
        "language",
        "languages",
        "profile",
        "qualification",
        "qualifications",
        "certificate",
        "certificates",
        "certifications",
        "certification",
        "information",
        "intern",
        "volunteer",
        "award",
        "awards",
        "jan",
        "feb",
        "mar",
        "apr",
        "may",
        "jun",
        "jul",
        "aug",
        "sep",
        "oct",
        "nov",
        "dec",
    ]
    token_lower = token.lemma_.lower()
    return all(
        [
            not token.is_space,  # not space
            not token.is_punct,  # not punct
            not token.is_bracket,  # not bracket
            not token.is_quote,  # not quote
            not token.is_currency,  # not currency
            not token.like_num,  # not number
            not token.like_url,  # not url
            not token.like_email,  # not email
            not token.is_oov,  # not out of vocab
            not token.is_stop,  # not a stopword
            token.is_alpha,  # is alphabetical
            token.has_vector,
            token_lower not in customized_stop_words,
            token.pos_ not in ["ADV", "ADJ", "INTJ", "PART", "PRON", "X"],
            token.ent_type_
            not in [
                "PERSON",
                "ORG",
                "DATE",
                "CARDINAL",
                "TIME",
            ],  # not amongst these categories
        ]
    )


In [ ]:
import os
import pandas as pd
import re
import spacy
import docx

from typing import *



def loadDefaultNLP(is_big: bool = True) -> Any:
    """
    Function to load the default SpaCy nlp model into self.nlp
    :param is_big: if True, uses a large vocab set, else a small one
    :returns: nlp: a SpaCy nlp model
    """

    def segment_on_newline(doc):
        for token in doc[:-1]:
            if token.text.endswith("\n"):
                doc[token.i + 1].is_sent_start = True
        return doc

    if is_big:
        nlp = spacy.load("en_core_web_lg")
    else:
        nlp = spacy.load("en_core_web_sm")
    nlp.add_pipe(segment_on_newline, before="parser")
    return nlp


def countWords(line: str) -> int:
    """
    Counts the numbers of words in a line
    :param line: line to count
    :return count: num of lines
    """
    count = 0
    is_space = False
    for c in line:
        is_not_char = not c.isspace()
        if is_space and is_not_char:
            count += 1
        is_space = not is_not_char
    return count


def getAllTokensAndChunks(doc) -> Tuple[Set[Any], Set[Any]]:
    """
    Converts a spacy doc into tokens and chunks. Tokens and chunks pass through a customFilter first
    :param doc: a SpaCy doc
    :returns: seen_chunks_words: set of strings seen
    :returns: all_tokens_chunks: set of all tokens and chunks found
    """
    # used to test duplicate words/chunks
    seen_chunks_words = set()
    # collate all words/chunks
    all_tokens_chunks = set()
    # generate all 1-gram tokens
    for token in doc:
        w = token.lemma_.lower()
        if (w not in seen_chunks_words) and customFilter(token):
            all_tokens_chunks.add(token)
            seen_chunks_words.add(w)

    # generate all n-gram tokens
    for chunk in doc.noun_chunks:
        c = chunk.lemma_.lower()
        if (
            len(chunk) > 1
            and (c not in seen_chunks_words)
            and all(customFilter(token) for token in chunk)
        ):
            all_tokens_chunks.add(chunk)
            seen_chunks_words.add(c)

    return seen_chunks_words, all_tokens_chunks


def findDocumentsRecursive(base_dir: str) -> Optional[List[str]]:
    """
    Recursively get all documents from `base_dir`
    :param base_dir: base directory of documents
    :returns out: a list of full file names of the documents
    """
    out: List[str] = []

    # check if base_dir is a proper dir
    if not os.path.isdir(base_dir):
        return None

    for d in os.listdir(base_dir):
        full_path = os.path.join(base_dir, d)
        if os.path.isdir(full_path):
            out.extend(findDocumentsRecursive(full_path))
        else:
            for end in (".pdf", ".docx"):
                if full_path.endswith(end):
                    out.append(full_path)
    return out


def generateDFFromData(
    data: Dict[Any, Any],
    filename: str,
    save_csv: bool = False
) -> pd.DataFrame:
    """
    Generates DF for model creation
    :param data: dictionary of data
    :param filename: what to save model as
    :param save_csv: whether to save the model as csv
    :returns data_df: the model df
    """
    data_df = pd.DataFrame(data=data)
    data_df.sort_values(by=["score"], ascending=False, inplace=True)
    data_df.reset_index(inplace=True)
    if save_csv:
        data_df.to_csv(filename)
    return data_df


def getDocxText(filename: str) -> str:
    """
    Get the text from a docx file
    :param filename: docx file
    :returns fullText: text of file
    """
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        txt = para.text
        fullText.append(txt)
    return "\n".join(fullText)


def getPDFText(filename: str, parser) -> str:
    """
    Get the text from a pdf file
    :param filename: pdf file
    :param parser: pdf parser
    :returns fullText: text of file
    """
    raw = parser.from_file(filename)
    new_text = raw["content"]
    if "title" in raw["metadata"]:
        title = raw["metadata"]["title"]
        new_text = new_text.replace(title, "")
    return new_text


def loadDocumentIntoSpacy(f: str, parser, spacy_nlp) -> Optional[Tuple[Any, str]]:
    """
    Convert file into spacy Document
    :param f: filename
    :param parser: pdf_parser
    :param spacy_nlp: nlp model
    :returns nlp_doc: nlp doc
    :returns new_text: text of file
    """
    if f.endswith(".pdf"):
        new_text = getPDFText(f, parser)
    elif f.endswith(".docx"):
        new_text = getDocxText(f)
    else:
        return None, None

    # new_text = "\n".join(
    #     [line.strip() for line in new_text.split("\n") if len(line) > 1]
    # )
    new_text = re.sub("\n{3,}", "\n", new_text)
    new_text = str(bytes(new_text, "utf-8").replace(b"\xe2\x80\x93", b""), "utf-8")
    # convert to spacy doc
    return spacy_nlp(new_text), new_text

In [ ]:
import re
import pandas as pd
import os
import sys

from collections import Counter, defaultdict
from datetime import datetime
from dateutil import relativedelta
from typing import *

WORDS_LIST = {
    "Work": ["(Work|WORK)", "(Experience(s?)|EXPERIENCE(S?))", "(History|HISTORY)"],
    "Education": ["(Education|EDUCATION)", "(Qualifications|QUALIFICATIONS)"],
    "Skills": [
        "(Skills|SKILLS)",
        "(Proficiency|PROFICIENCY)",
        "LANGUAGE",
        "CERTIFICATION",
    ],
    "Projects": ["(Projects|PROJECTS)"],
    "Activities": ["(Leadership|LEADERSHIP)", "(Activities|ACTIVITIES)"],
}

main_skills = ""

class InfoExtractor:
    """
    Extracts key information from resumes
    """

    def __init__(self, spacy_nlp_model, parser):
        self.nlp = spacy_nlp_model
        self.parser = parser

    def extractFromFile(self, filename):
        doc, text = loadDocumentIntoSpacy(filename, self.parser, self.nlp)
        skills = self.extractFromText(doc, text, filename)
        return skills

    def extractFromText(self, doc, text, filename):
        name = InfoExtractor.findName(doc, filename)
        if name is None:
            name = ""
        email = InfoExtractor.findEmail(doc)
        if email is None:
            email = ""
        number = InfoExtractor.findNumber(doc)
        if number is None:
            number = ""
        city = InfoExtractor.findCity(doc)
        if city is None:
            city = ""
        categories = InfoExtractor.extractCategories(text)
        workAndEducation = InfoExtractor.findWorkAndEducation(
            categories, doc, text, name
        )
        totalWorkExperience = InfoExtractor.getTotalExperienceFormatted(
            workAndEducation["Work"]
        )
        totalEducationExperience = InfoExtractor.getTotalExperienceFormatted(
            workAndEducation["Education"]
        )
        allSkills = ", ".join(InfoExtractor.extractSkills(doc))
        
        #print("\nWork Experience:")
        #print(totalWorkExperience)
        #for w in workAndEducation["Work"]:
            #print(" - " + w)
        #print("\nEducation:")
        #print(totalEducationExperience)
        #for e in workAndEducation["Education"]:
            #print(" - " + e)
        #print("\nSkills:")
        #print(allSkills)
        
        return [allSkills,totalWorkExperience]
        

    @staticmethod
    def extractSkills(doc) -> List[str]:
        """
        Helper function to extract skills from spacy nlp text

        :param doc: object of `spacy.tokens.doc.Doc`
        :return: list of skills extracted
        """
        #print(doc)
        tokens = [token.text for token in doc if not token.is_stop]
        data = pd.read_csv("./skills.csv")
    
        skills = list(data.columns.values)
        skillset = []
        # check for one-grams
        for token in tokens:
            if token.lower() in skills:
                skillset.append(token)

        # check for bi-grams and tri-grams
        for token in doc.noun_chunks:
            token = token.text.lower().strip()
            if token in skills:
                skillset.append(token)
        return [i.capitalize() for i in set([i.lower() for i in skillset])]

    @staticmethod
    def extractCategories(text) -> Dict[str, List[Tuple[int, int]]]:
       
        data = defaultdict(list)
        page_count = 0
        prev_count = 0
        prev_line = None
        prev_k = None
        for line in text.split("\n"):
            line = re.sub(r"\s+?", " ", line).strip()
            for (k, wl) in WORDS_LIST.items():
                # for each word in the list
                for w in wl:
                    # if category has not been found and not a very long line
                    # - long line likely not a category
                    if countWords(line) < 10:
                        match = re.findall(w, line)
                        if match:
                            size = page_count - prev_count
                            # append previous
                            if prev_k is not None:
                                data[prev_k].append((size, prev_count, prev_line))
                            prev_count = page_count
                            prev_k = k
                            prev_line = line
            page_count += 1

        # last item
        if prev_k is not None:
            size = page_count - prev_count - 1 # -1 cuz page_count += 1 on prev line
            data[prev_k].append((size, prev_count, prev_line))

        # choose the biggest category (reduce false positives)
        for k in data:
            if len(data[k]) >= 2:
                data[k] = [max(data[k], key=lambda x: x[0])]
        return data

    @staticmethod
    def findWorkAndEducation(categories, doc, text, name) -> Dict[str, List[str]]:
        inv_data = {v[0][1]: (v[0][0], k) for k, v in categories.items()}
        line_count = 0
        exp_list = defaultdict(list)
        name = name.lower()

        current_line = None
        is_dot = False
        is_space = True
        continuation_sent = []
        first_line = None
        unique_char_regex = "[^\sA-Za-z0-9\.\/\(\)\,\-\|]+"

        for line in text.split("\n"):
            line = re.sub(r"\s+", " ", line).strip()
            match = re.search(r"^.*:", line)
            if match:
                line = line[match.end() :].strip()

            # get first non-space line for filtering since
            # sometimes it might be a page header
            if line and first_line is None:
                first_line = line

            # update line_countfirst since there are `continue`s below
            line_count += 1
            if (line_count - 1) in inv_data:
                current_line = inv_data[line_count - 1][1]
            # contains a full-blown state-machine for filtering stuff
            elif current_line == "Work":
                if line:
                    # if name is inside, skip
                    if name == line:
                        continue
                    # if like first line of resume, skip
                    if line == first_line:
                        continue
                    # check if it's not a list with some unique character as list bullet
                    has_dot = re.findall(unique_char_regex, line[:5])
                    # if last paragraph is a list item
                    if is_dot:
                        # if this paragraph is not a list item and the previous line is a space
                        if not has_dot and is_space:
                            if line[0].isupper() or re.findall(r"^\d+\.", line[:5]):
                                exp_list[current_line].append(line)
                                is_dot = False

                    else:
                        if not has_dot and (
                            line[0].isupper() or re.findall(r"^\d+\.", line[:5])
                        ):
                            exp_list[current_line].append(line)
                            is_dot = False
                    if has_dot:
                        is_dot = True
                    is_space = False
                else:
                    is_space = True
            elif current_line == "Education":
                if line:
                    # if not like first line
                    if line == first_line:
                        continue
                    line = re.sub(unique_char_regex, '', line[:5]) + line[5:]
                    if len(line) < 12:
                        continuation_sent.append(line)
                    else:
                        if continuation_sent:
                            continuation_sent.append(line)
                            line = " ".join(continuation_sent)
                            continuation_sent = []
                        exp_list[current_line].append(line)

        return exp_list

    @staticmethod
    def findNumber(doc) -> Optional[str]:
        """
        Helper function to extract number from nlp doc
        :param doc: SpaCy Doc of text
        :return: int:number if found, else None
        """
        for sent in doc.sents:
            num = re.findall(r"\(?\+?\d+\)?\d+(?:[- \)]+\d+)*", sent.text)
            if num:
                for n in num:
                    if len(n) >= 8 and (
                        not re.findall(r"^[0-9]{2,4} *-+ *[0-9]{2,4}$", n)
                    ):
                        return n
        return None

    @staticmethod
    def findEmail(doc) -> Optional[str]:
        """
        Helper function to extract email from nlp doc
        :param doc: SpaCy Doc of text
        :return: str:email if found, else None
        """
        for token in doc:
            if token.like_email:
                return token.text
        return None

    @staticmethod
    def findCity(doc) -> Optional[str]:
        counter = Counter()
        """
        Helper function to extract most likely City/Country from nlp doc
        :param doc: SpaCy Doc of text
        :return: str:city/country if found, else None
        """
        for ent in doc.ents:
            if ent.label_ == "GPE":
                counter[ent.text] += 1

        if len(counter) >= 1:
            return counter.most_common(1)[0][0]
        return None

    @staticmethod
    def findName(doc, filename) -> Optional[str]:
        """
        Helper function to extract name from nlp doc
        :param doc: SpaCy Doc of text
        :param filename: used as backup if NE cannot be found
        :return: str:NAME_PATTERN if found, else None
        """
        to_chain = False
        all_names = []
        person_name = None

        for ent in doc.ents:
            if ent.label_ == "PERSON":
                if not to_chain:
                    person_name = ent.text.strip()
                    to_chain = True
                else:
                    person_name = person_name + " " + ent.text.strip()
            elif ent.label_ != "PERSON":
                if to_chain:
                    all_names.append(person_name)
                    person_name = None
                    to_chain = False
        if all_names:
            return all_names[0]
        else:
            try:
                base_name_wo_ex = os.path.splitext(os.path.basename(filename))[0]
                return base_name_wo_ex + " (from filename)"
            except:
                return None

    @staticmethod
    def getNumberOfMonths(datepair) -> int:
        """
        Helper function to extract total months of experience from a resume

        :param date1: Starting date
        :param date2: Ending date
        :return: months of experience from date1 to date2
        """
        # if years
        # if years
        date2_parsed = False
        if datepair.get("fh", None) is not None:
            gap = datepair["fh"]
        else:
            gap = ""
        try:
            present_vocab = ("present", "date", "now")
            if "syear" in datepair:
                date1 = datepair["fyear"]
                date2 = datepair["syear"]

                if date2.lower() in present_vocab:
                    date2 = datetime.now()
                    date2_parsed = True

                try:
                    if not date2_parsed:
                        date2 = datetime.strptime(str(date2), "%Y")
                    date1 = datetime.strptime(str(date1), "%Y")
                except:
                    pass
            elif "smonth_num" in datepair:
                date1 = datepair["fmonth_num"]
                date2 = datepair["smonth_num"]

                if date2.lower() in present_vocab:
                    date2 = datetime.now()
                    date2_parsed = True

                for stype in ("%m" + gap + "%Y", "%m" + gap + "%y"):
                    try:
                        if not date2_parsed:
                            date2 = datetime.strptime(str(date2), stype)
                        date1 = datetime.strptime(str(date1), stype)
                        break
                    except:
                        pass
            else:
                date1 = datepair["fmonth"]
                date2 = datepair["smonth"]

                if date2.lower() in present_vocab:
                    date2 = datetime.now()
                    date2_parsed = True

                for stype in (
                    "%b" + gap + "%Y",
                    "%b" + gap + "%y",
                    "%B" + gap + "%Y",
                    "%B" + gap + "%y",
                ):
                    try:
                        if not date2_parsed:
                            date2 = datetime.strptime(str(date2), stype)
                        date1 = datetime.strptime(str(date1), stype)
                        break
                    except:
                        pass

            months_of_experience = relativedelta.relativedelta(date2, date1)
            months_of_experience = (
                months_of_experience.years * 12 + months_of_experience.months
            )
            return months_of_experience
        except Exception as e:
            return 0

    @staticmethod
    def getTotalExperience(experience_list) -> int:
        """
        Wrapper function to extract total months of experience from a resume

        :param experience_list: list of experience text extracted
        :return: total months of experience
        """
        exp_ = []
        for line in experience_list:
            line = line.lower().strip()
            # have to split search since regex OR does not capture on a first-come-first-serve basis
            experience = re.search(
                r"(?P<fyear>\d{4})\s*(\s|-|to)\s*(?P<syear>\d{4}|present|date|now)",
                line,
                re.I,
            )
            if experience:
                d = experience.groupdict()
                exp_.append(d)
                continue

            experience = re.search(
                r"(?P<fmonth>\w+(?P<fh>.)\d+)\s*(\s|-|to)\s*(?P<smonth>\w+(?P<sh>.)\d+|present|date|now)",
                line,
                re.I,
            )
            if experience:
                d = experience.groupdict()
                exp_.append(d)
                continue

            experience = re.search(
                r"(?P<fmonth_num>\d+(?P<fh>.)\d+)\s*(\s|-|to)\s*(?P<smonth_num>\d+(?P<sh>.)\d+|present|date|now)",
                line,
                re.I,
            )
            if experience:
                d = experience.groupdict()
                exp_.append(d)
                continue
        experience_num_list = [InfoExtractor.getNumberOfMonths(i) for i in exp_]
        total_experience_in_months = sum(experience_num_list)
        return total_experience_in_months

    @staticmethod
    def getTotalExperienceFormatted(exp_list) -> str:
        months = InfoExtractor.getTotalExperience(exp_list)
        if months < 12:
            return str(months) + " months"
        years = months // 12
        months = months % 12
        return str(years) + " years " + str(months) + " months"







In [ ]:
pip install sklearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

resumeList = ['./kush.pdf','./marco.pdf','./nava.pdf','./parth.pdf','./pulkit.pdf', './ayesha.pdf','./rakshith.pdf']
main_list = []

def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele
        str1 += " "   
    
    # return string   
    return str1

for i in resumeList:
 
  filename = i
  from tika import parser

  nlp = loadDefaultNLP(is_big=False)
  infoExtractor = InfoExtractor(nlp, parser)
  skills = infoExtractor.extractFromFile(filename)

  job_desc = "Google HTML CSS React.js Hard-Working Anguar.js JavaScript Software Engineer API"

  #ALL CATEGORIES

  text = [skills[0],job_desc]

  cv = CountVectorizer()

  count_matrix = cv.fit_transform(text)

  from sklearn.metrics.pairwise import cosine_similarity

  match_percent = cosine_similarity(count_matrix)[0][1]*100
  match_percent = round(match_percent,2)

  if('years' in skills[1]):
    main_list.append([filename,match_percent,int(skills[1][0])*12 + int(skills[1][8])])


  elif('months' in skills[1]):
    main_list.append([filename,match_percent,int(skills[1][0])])

  #print( filename + " This resume matches about " + str(match_percent)  + " percent to the job profile. \n" ) 

#print(main_list)
main_list.sort(key = lambda x: x[1])
main_list.reverse()
for i in main_list:
  print(i)

  